## Cryptography Talk: School for Advanced Studies - Wolfson Campus 

__Burton Rosenberg__
- University of Miami
- 25 May 2021

Talk given the the School of Advanced studies, the Wolfson Campus, at the request of Ms. Daire, mathematics instructor.

The webpage companion is [here](https://www.cs.miami.edu/home/burt/learning/wolfson-talk/)

In [45]:
def make_simple_key(secret):
    """
    turns a secret to a key.
    
    both the secret and the key are strings.
    secrets can be any case.
    the key is in uppercase
    
    starts by adding each unique letter from secret to the key, then then filling it out
    with the unused letters in alphabetical order.
    """
    key = ''
    for letter in secret:
        letter = letter.upper()
        if letter not in key:
            key += letter
    for letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
        if letter not in key:
            key += letter
    return key

def simple_subst_encrypt(key, plaintext):
    """
    takes a key created by make_simple_key and a plaintext and returns the encrypted text
    the returned cipher text is as traditional in uppercase.
    """
    ciphertext = ''
    for letter in plaintext:
        if letter.isalpha():
            i = ord(letter.upper()) - ord('A')
            ciphertext += key[i]
    return ciphertext

def simple_subst_decrypt(key, ciphertext):
    """
    takes a key used in the simple_subst_encrypt and a ciphertext and returns the deciphered text.
    the ciphertext is assumed to be uppercase.
    the returned plaintext is as traditional in lowercase.
    """
    plaintext = ''
    for letter in ciphertext:
        i = key.find(letter)
        plaintext += chr(i+ord('a'))
    return plaintext


def count_letters(the_text):
    count_dict = {}
    for letter in the_text:
        if letter.isalpha() :
            letter = letter.lower()
            if letter in count_dict:
                count_dict[letter] += 1  # add to the count
            else:
                count_dict[letter] = 1   # congrat's, the letter appeared
    sorted_dict = {}
    for letter in sorted(count_dict, key=count_dict.get, reverse=True):
        sorted_dict[letter] = count_dict[letter]
    return sorted_dict
        


In [46]:
    
key = make_simple_key('wolfsoncampus')
plaintext = """
the world is a vampire
"""
ciphertext = simple_subst_encrypt(key,plaintext)
print(ciphertext)
print(simple_subst_decrypt(key,ciphertext))

sonnet_18 = """
shall i compare thee to a summer's day
Thou art more lovely and more temperate:
Rough winds do shake the darling buds of May,
And summer’s lease hath all too short a date;
Sometime too hot the eye of heaven shines,
And often is his gold complexion dimm'd;
And every fair from fair sometime declines,
By chance or nature’s changing course untrimm'd;
But thy eternal summer shall not fade,
"""

print(count_letters(sonnet_18))
sonnet_18_encrypted = simple_subst_encrypt(key,sonnet_18)
print(count_letters(sonnet_18_encrypted))


QASVGJBFMKWTWDHMJS
theworldisavampire
{'e': 39, 'a': 27, 'o': 26, 't': 23, 's': 21, 'm': 21, 'r': 20, 'h': 18, 'n': 18, 'd': 16, 'i': 15, 'l': 14, 'u': 10, 'c': 7, 'y': 7, 'f': 7, 'g': 5, 'p': 3, 'v': 3, 'b': 3, 'w': 1, 'k': 1, 'x': 1}
{'s': 39, 'w': 27, 'g': 26, 'q': 23, 'k': 21, 'd': 21, 'j': 20, 'a': 18, 'e': 18, 'f': 16, 'm': 15, 'b': 14, 'r': 10, 'l': 7, 'y': 7, 'n': 7, 'c': 5, 'h': 3, 't': 3, 'o': 3, 'v': 1, 'u': 1, 'x': 1}


In [137]:
def make_simple_key_playfair(secret):
    """
    turns a secret to a key for a playfair cipher
    
    same was for a simple substitution, except I and J are treated as the same letter
    
    both the secret and the key are strings.
    secrets can be any case.
    the key is in uppercase
    
    starts by adding each unique letter from secret to the key, then then filling it out
    with the unused letters in alphabetical order.
    """
    key = ''
    for letter in secret:
        letter = letter.upper()
        if letter == 'J':
            letter = 'I'
        if letter not in key:
            key += letter
    for letter in 'ABCDEFGHIKLMNOPQRSTUVWXYZ':
        if letter not in key:
            key += letter
    return key

def print_playfair_key(key):
    box_string = '+---+---+---+---+---+'
    first = True
    for i, letter in enumerate(key):
        if i%5 == 0:
            if first:
                first = False
            else:
                print('|')
            print(box_string)
        print(f'| {letter} ', end='')
        i += 1
    print('|')
    print(box_string)


def playfair_encrypt(key, plaintext, encrypt=True):
    
    def find_key(key, letter):
        i  = key.find(letter)
        return (i//5,i%5)
    
    def key_to_letter(key,row_col):
        row, col = row_col
        return key[5*row+col]
    
    def enc_rules(rc1, rc2, enc=True):
        if enc:
            rule_dir = 1
        else:
            rule_dir = -1
    
        if rc1==rc2:
            print('warning! double letter will reveal key.\n\tEncrypting as a constant.')
            return [ (4,4),(4,4) ]
        
        if rc1[0]==rc2[0]:
            # same row
            return [ (rc1[0],(rc1[1]+rule_dir)%5), (rc2[0],(rc2[1]+rule_dir)%5) ]
        if rc1[1]==rc2[1]:
            # same col
            return [((rc1[0]+rule_dir)%5,rc1[1]), ((rc2[0]+rule_dir)%5,rc2[1])]
        # square
        return [ (rc1[0],rc2[1]), (rc2[0],rc1[1])  ]
        
    pt = ''
    for letter in plaintext:
        if letter.isalpha():
            pt += letter.upper()
    if len(pt)%2 != 0:
        pt += 'X'
    
    ct = ''
    in_letters = [(0,0),(0,0)]

    for l1, l2 in zip(pt[0::2],pt[1::2]):
        rc1 = find_key(key,l1)
        rc2 = find_key(key,l2)
        rco1, rco2 = enc_rules(rc1, rc2, encrypt)
        ct += key_to_letter(key,rco1)
        ct += key_to_letter(key,rco2)
        ct += ' '
    
    if not encrypt:
        return ct.lower()
    return ct

In [143]:
key = make_simple_key_playfair('wolfsoncampus')
print_playfair_key(key)

print(playfair_encrypt(key,'wclmmlwllsfslddleyyq'))

mystery = playfair_encrypt(key,'the world is a vampire')
print(mystery)
print( playfair_encrypt(key,mystery,encrypt=False))

+---+---+---+---+---+
| W | O | L | F | S |
+---+---+---+---+---+
| N | C | A | M | P |
+---+---+---+---+---+
| U | B | D | E | G |
+---+---+---+---+---+
| H | I | K | Q | R |
+---+---+---+---+---+
| T | V | X | Y | Z |
+---+---+---+---+---+
ON FA AF OF FW SW AK KA QF FY 
WT UF SI AK RO CX MP CR QG 
th ew or ld is av am pi re 
